In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,25681
2,Huelva,Confirmados PDIA 14 días,695
3,Huelva,Tasa PDIA 14 días,"135,4327026131691"
4,Huelva,Confirmados PDIA 7 días,367
5,Huelva,Total Confirmados,25882
6,Huelva,Curados,21083
7,Huelva,Fallecidos,351


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  25681.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6721.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 2480 personas en los últimos 7 días 

Un positivo PCR cada 1547 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,25681.0,367.0,695.0,513170.0,71.516262,135.432703,73.0
Condado-Campiña,7499.0,238.0,436.0,156231.0,152.338524,279.073935,52.0
Huelva-Costa,15430.0,109.0,221.0,289548.0,37.644881,76.325860,23.0
Huelva (capital),6721.0,58.0,93.0,143837.0,40.323422,64.656521,14.0
Bonares,196.0,15.0,16.0,6060.0,247.524752,264.026403,10.0
Almonte,1028.0,57.0,126.0,24507.0,232.586608,514.138817,8.0
San Juan del Puerto,379.0,9.0,13.0,9411.0,95.632770,138.136224,7.0
Rociana del Condado,381.0,31.0,60.0,7939.0,390.477390,755.762691,7.0
Moguer,894.0,23.0,45.0,21867.0,105.181323,205.789546,5.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Rociana del Condado,381.0,31.0,60.0,7939.0,390.477390,755.762691,7.0
San Bartolomé de la Torre,237.0,17.0,27.0,3761.0,452.007445,717.894177,3.0
Alosno,150.0,2.0,25.0,3933.0,50.851767,635.647089,0.0
Almonte,1028.0,57.0,126.0,24507.0,232.586608,514.138817,8.0
Niebla,175.0,8.0,17.0,4116.0,194.363460,413.022352,0.0
Bollullos Par del Condado,683.0,33.0,52.0,14387.0,229.373740,361.437409,3.0
Condado-Campiña,7499.0,238.0,436.0,156231.0,152.338524,279.073935,52.0
Bonares,196.0,15.0,16.0,6060.0,247.524752,264.026403,10.0
Paterna del Campo,120.0,2.0,8.0,3457.0,57.853630,231.414521,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Alosno,150.0,2.0,25.0,3933.0,50.851767,635.647089,0.0,0.080000
Isla Cristina,2012.0,2.0,12.0,21393.0,9.348852,56.093115,1.0,0.166667
Paterna del Campo,120.0,2.0,8.0,3457.0,57.853630,231.414521,0.0,0.250000
Ayamonte,1245.0,5.0,12.0,21104.0,23.692191,56.861259,1.0,0.416667
Gibraleón,546.0,10.0,24.0,12737.0,78.511423,188.427416,0.0,0.416667
Villalba del Alcor,462.0,3.0,7.0,3366.0,89.126560,207.961973,0.0,0.428571
Valverde del Camino,430.0,3.0,7.0,12750.0,23.529412,54.901961,1.0,0.428571
Punta Umbría,667.0,4.0,9.0,15355.0,26.050147,58.612830,1.0,0.444444
Almonte,1028.0,57.0,126.0,24507.0,232.586608,514.138817,8.0,0.452381
